In [7]:
import parameter
import query
import time
from model import Model
from station import Station
import numpy as np

In [8]:
now = time.time()
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 6, 30, 0, 0, 0)
station = parameter.fast_station(station='E 7 St & Ave B', start_date=start_date, end_date=end_date)
print(time.time() - now)


11.156305074691772


In [12]:
for i in range(24):
    print(np.random.choice(station.neighbors_names, 5, p=station.transition[i]))

['Pioneer St & Van Brunt St' 'E 7 St & Ave B' 'E 7 St & Ave B'
 '9 Ave & W 18 St' 'St. Johns Pl & Kingston Ave']
['Broadway & E 14 St' 'E 7 St & Ave B' 'E 7 St & Avenue A'
 'FDR Drive & E 35 St' 'Broadway & E 14 St']
['E 7 St & Ave B' 'McKibbin St & Manhattan Ave' 'E 6 St & 2 Ave'
 'McKibbin St & Manhattan Ave' 'McKibbin St & Manhattan Ave']
['Centre St & Worth St' 'E 6 St & Avenue D' 'Avenue D & E 10 St'
 'Lincoln Ave & E 134 St' 'Lincoln Ave & E 134 St']
['Canal St & Rutgers St' 'E 6 St & Avenue D' 'Clinton St & Grand St'
 'E 10 St & Avenue A' 'E 10 St & Avenue A']
['E 2 St & Avenue A' 'Cooper Square & Astor Pl' 'E 6 St & Avenue D'
 'Bleecker St & Crosby St' 'Cooper Square & Astor Pl']
['E 7 St & Ave B' 'Stanton St & Norfolk St' 'Cleveland Pl & Spring St'
 'Cooper Square & Astor Pl' 'W 13 St & 5 Ave']
['E 47 St & Park Ave' 'Ave A & E 14 St' 'MacDougal St & Prince St'
 'Cooper Square & Astor Pl' 'Canal St & Rutgers St']
['Ave A & E 14 St' 'E 51 St & Lexington Ave' 'Ave A & E 14 St'
 '

In [2]:
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 6, 30, 0, 0, 0)
tph = 1

whitelist_stations = ['E 7 St & Ave B',
                      'Cooper Square & Astor Pl',
                      'E 7 St & Ave C',
                      'Ave A & E 14 St',
                      'W 21 St & 6 Ave']

station_list = {}
for station in whitelist_stations:
    rate = parameter.get_rate(station=station, 
                              start_date=start_date, 
                              end_date=end_date, 
                              tph=tph)
    transition_data = parameter.get_transition(station=station, 
                                               start_date=start_date, 
                                               end_date=end_date, 
                                               tph=tph)
    transition_stations = transition_data[0]
    neighbors_dist = {}
    for dest_station in transition_stations:
        neighbors_dist[dest_station] = parameter.avg_travel_time(orig_station=station, 
                                                            dest_station=dest_station)
    transition = transition_data[1]
    max_docks = 30
    curr_bikes = 15
    station_list[station] = Station(name=station, 
                                    id = 0,
                                    neighbors_dist=neighbors_dist,
                                    neighbors_names=transition_stations,
                                    rate=rate,
                                    transition=transition, 
                                    max_docks=max_docks, 
                                    curr_bikes=curr_bikes,)

In [3]:
model = Model(station_names=whitelist_stations, stations_dict=station_list, in_transit=[], tph=tph)

In [13]:
for station in model.stations_dict.values():
    print(station.name, 'has', station.curr_bikes, 'bikes')
print(len(model.in_transit), 'bikes in transit')
print()
for trip in model.in_transit:
    print(trip.start_station, 'to', trip.end_station)
    print(trip.start_time, 'to', trip.end_time, 'currently', trip.curr_time)

E 7 St & Ave B has 12 bikes
Cooper Square & Astor Pl has 13 bikes
E 7 St & Ave C has 15 bikes
Ave A & E 14 St has 11 bikes
W 21 St & 6 Ave has 14 bikes
10 bikes in transit

E 7 St & Ave B to E 7 St & Ave B
5:00:00 to 0 days 05:08:00.751168224 currently 5:00:00
E 7 St & Ave B to E 7 St & Ave B
5:00:00 to 0 days 05:08:00.751168224 currently 5:00:00
Cooper Square & Astor Pl to Cooper Square & Astor Pl
5:00:00 to 0 days 05:05:33.699511094 currently 5:00:00
Cooper Square & Astor Pl to Cooper Square & Astor Pl
5:00:00 to 0 days 05:05:33.699511094 currently 5:00:00
Cooper Square & Astor Pl to Cooper Square & Astor Pl
5:00:00 to 0 days 05:05:33.699511094 currently 5:00:00
E 7 St & Ave C to E 7 St & Ave C
5:00:00 to 0 days 05:05:42.451834862 currently 5:00:00
Ave A & E 14 St to Ave A & E 14 St
5:00:00 to 0 days 05:06:33.995951417 currently 5:00:00
Ave A & E 14 St to Ave A & E 14 St
5:00:00 to 0 days 05:06:33.995951417 currently 5:00:00
Ave A & E 14 St to Ave A & E 14 St
5:00:00 to 0 days 05:06:

In [14]:
unique_stations = ['test', 'test2', 'test3', 'test4', 'test5', 'test6', 'test7', 'test8']

test = [{station: 0 for station in unique_stations} for i in range(24)]
test[0]['test'] += 3
test


[{'test': 3,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test8': 0},
 {'test': 0,
  'test2': 0,
  'test3': 0,
  'test4': 0,
  'test5': 0,
  'test6': 0,
  'test7': 0,
  'test

In [7]:
[0]*5

[0, 0, 0, 0, 0]

In [2]:
station_1 = Station(
    name='Station 1', 
    id=1, 
    neighbors_dist={'Station 1': 0, 'Station 2': 1, 'Station 3': 3, 'Station 4': 2, 'Station 5': 7}, 
    nearest_neighbors=[], 
    max_docks=30, 
    curr_bikes=15, 
    full=False,
    empty=False,
    rate = [abs(np.sin(np.pi*i+1)*1.5) for i in range(24)],
    transition=[0, 0.15, 0.1, 0.25, 0.5])

station_2 = Station(
    name='Station 2', 
    id=2, 
    neighbors_dist={'Station 1': 1, 'Station 2': 0, 'Station 3': 4, 'Station 4': 1, 'Station 5': 8}, 
    nearest_neighbors=[], 
    max_docks=30, 
    curr_bikes=10, 
    full=False,
    empty=False,
    rate = [abs(np.sin(np.pi*i+2)*2) for i in range(24)],
    transition=[0.4, 0, 0.1, 0.25, 0.25])

station_3 = Station(
    name='Station 3', 
    id=3, 
    neighbors_dist={'Station 1': 3, 'Station 2': 4, 'Station 3': 0, 'Station 4': 1, 'Station 5': 4}, 
    nearest_neighbors=[], 
    max_docks=45, 
    curr_bikes=25, 
    full=False,
    empty=False,
    rate = [abs(np.sin(np.pi*i+3)) for i in range(24)],
    transition=[0.5, 0.2, 0, 0.25, 0.05])

station_4 = Station(
    name='Station 4', 
    id=1, 
    neighbors_dist={'Station 1': 2, 'Station 2': 1, 'Station 3': 1, 'Station 4': 0, 'Station 5': 5}, 
    nearest_neighbors=[], 
    max_docks=30, 
    curr_bikes=15, 
    full=False,
    empty=False,
    rate = [abs(np.sin(np.pi*i/12)*2) for i in range(24)],
    transition=[0.25, 0.05, 0.2, 0, 0.5])

station_5 = Station(
    name='Station 5', 
    id=1, 
    neighbors_dist={'Station 1': 7, 'Station 2': 8, 'Station 3': 4, 'Station 4': 5, 'Station 5': 0}, 
    nearest_neighbors=[], 
    max_docks=30, 
    curr_bikes=22, 
    full=False,
    empty=False,
    rate = [abs(np.sin(np.pi*i+1)*2) for i in range(24)],
    transition=[0.6, 0.05, 0.1, 0.25, 0])


station_list = ['Station 1', 'Station 2', 'Station 3', 'Station 4', 'Station 5']
stations = {'Station 1': station_1, 'Station 2': station_2, 'Station 3': station_3, 'Station 4': station_4, 'Station 5': station_5}
in_transit = []

model = Model(station_list, stations, in_transit)

In [9]:
model.sim(1)

for station_name in model.stations:
    station = model.stations[station_name]
    print(station.name, 'rate', round(station.rate[model.curr_time], 3))
    print(station.curr_bikes, 'bikes in stock')
    print()
    
print(len(model.in_transit), 'in transit')

Failure to depart
Station 2 to Station 1 in 1 with 0 completed
Failure to depart
Station 2 to Station 5 in 8 with 0 completed
Station 1 rate 1.262
11 bikes in stock

Station 2 rate 1.819
0 bikes in stock

Station 3 rate 0.141
27 bikes in stock

Station 4 rate 1.932
1 bikes in stock

Station 5 rate 1.683
13 bikes in stock

35 in transit
